In [7]:
# import tensorflow as tf
# import pandas as pd
import numpy as np
import pickle

data_destination = "./../MoviaFTP/"


def load_data(fp = '', pickle = False):
    data = np.load(fp, allow_pickle = pickle )
    return data

def transform_matrix_fwd(matrix):
    """
    do transformation for the forward route:
    'KDST' 'EGEV' 'HHLS' 'HHM' 'GLHO' 'NÆST' 'KLBV' 'RYST' 'HKP' 'NPST1'
    """
    link_counts = []
    for i in range(len(matrix)-1):
        sum = np.sum(matrix[0:i+1, i+1:])
        link_counts.append(sum)
    return link_counts

def transform_matrix_bwd(matrix):
    """
    do a transformation for the reverse route:
    'NPST1' 'HKP' 'RYST' 'KLBV' 'NÆST' 'GLHO' 'HHM' 'HHLS' 'EGEV' 'KDST' 
    """
    link_counts = []
    for i in range(len(matrix)-1):
        sum = np.sum(matrix[i+1:, 0:i+1])
        link_counts.append(sum)
    link_counts.reverse()
    return link_counts


def transform_all(matrices):
    """Returns: tuple (fwd,bwd) of the route counts of the matrices
    """
    all_counts_fwd = []
    all_counts_bwd = []
    for i in range(len(matrices)):
        all_counts_fwd.append(transform_matrix_fwd(matrices[i]))
        all_counts_bwd.append(transform_matrix_bwd(matrices[i]))
    return (all_counts_fwd, all_counts_bwd)   


demand_matrices = load_data(data_destination + 'od-demand-202010-150.npy')

stop_names = load_data(data_destination + 'od_stop-202010-150.npy', pickle = True)
print("bus stops: ", stop_names[0:10])


dates = load_data(data_destination + 'od_time-202010-150.npy')
dates.shape = [np.shape(dates)[0], 1]
print("dimensions of date data: ", np.shape(dates))

# Finally we transform the data from matrices to link counts
route_counts = transform_all(demand_matrices)
print("dimension of transformed matrices: ", np.shape(route_counts[0]))

test = transform_matrix_fwd(demand_matrices[5700])
print("test: ", test)
print(demand_matrices[5700])

SyntaxError: invalid syntax (<ipython-input-7-6bb1591447ed>, line 6)